In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

ID=pd.read_excel('G:\Priyabrata\ML\Individual data.xlsx','SS+SS+IM')
print(ID)

     Sl no         Composition  Mixing entropy  Mixing enthalpy   Omega  \
0        1            FeCrNiAl           11.53           -21.02  1.5660   
1        2      Fe35(CrNiAl)65           11.32           -11.50  1.6500   
2        3      Fe45(CrNiAl)55           10.47            -9.81  1.8640   
3        4      Fe55(CrNiAl)45            9.83            -8.09  2.0910   
4        5      Fe65(CrNiAl)35            8.58            -6.35  2.3540   
..     ...                 ...             ...              ...     ...   
197    198   Nb31Ti30Zr26Al6V7           11.88            -7.57  3.4500   
198    199   Nb31Ti28Zr26Al6V9           12.10            -7.68  3.4700   
199    200  Nb31Ti26Zr26Al6V11           12.26            -7.78  3.4793   
200    201              MoReRu            9.13            -9.79  2.7870   
201    202    Ti10Fe30Co30Ni30           10.92           -10.26  1.9010   

      Gamma  Atomic size mismatch  Lambda    VEC  Pauling electronegativity  \
0    1.1575         

In [11]:
INPUT = ID.iloc[:,2:11]  #all INPUT (Variables)
INPUT

TARGET = ID.iloc[:,13]
TARGET

Y = ID.iloc[:,14] #coded phases. 0:MSS, 1:SSS
Y


0      1
1      1
2      1
3      1
4      1
      ..
197    0
198    0
199    0
200    0
201    1
Name: Code , Length: 202, dtype: int64

In [75]:
selected_INPUT = INPUT.drop(['Mixing entropy','Gamma','Pauling electronegativity','Molar volume dispersity'],axis=1)
selected_INPUT
normalized_INPUT=(selected_INPUT-selected_INPUT.mean())/selected_INPUT.std()

In [76]:
#ANN classifier for supervised learning
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn import metrics

#Repeated Stratified k fold cross validation (for imbalanced datasets)
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score


    
normalized_INPUT=(selected_INPUT-selected_INPUT.mean())/selected_INPUT.std()
nskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=1)
model2 = MLPClassifier(hidden_layer_sizes=(10,10), activation='relu', solver='adam', alpha=0.0000001, batch_size=15, learning_rate='adaptive', learning_rate_init=0.01,max_iter=1500, shuffle=True, warm_start='False') #batch size has no meaning for lbfgs
    
# evaluate model
scores = cross_val_score(model2, normalized_INPUT, Y, scoring='f1_macro', cv=nskf, verbose=3,n_jobs=-1)
scores
print('f1 score: %.3f (%.3f)' % (mean(scores), std(scores)))
print('\nMaximum f1 score: ', max(scores))
print('\nMinimum f1 score: ', min(scores))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    4.7s


f1 score: 0.855 (0.059)

Maximum f1 score:  1.0

Minimum f1 score:  0.6875


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    9.3s finished


In [78]:
#KNN

from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score

nskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=1)
model = KNeighborsClassifier(metric='minkowski',metric_params = None, n_jobs=-1, n_neighbors=5, p=2,weights='uniform')
scores = cross_val_score(model, normalized_INPUT, Y, scoring='f1_macro', cv=nskf, n_jobs=-1)
scores
    
# evaluate model
scores = cross_val_score(model, normalized_INPUT, Y, scoring='f1_macro', cv=nskf, verbose=3,n_jobs=-1)
scores
print('f1 score: %.3f (%.3f)' % (mean(scores), std(scores)))
print('\nMaximum f1 score: ', max(scores))
print('\nMinimum f1 score: ', min(scores))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.3s


f1 score: 0.835 (0.070)

Maximum f1 score:  1.0

Minimum f1 score:  0.6307692307692307


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    1.0s finished


In [71]:
#RF
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std
import matplotlib.pyplot as plt

rfc = RandomForestClassifier(n_estimators=50, max_depth=2, max_leaf_nodes=4, min_samples_split=6,min_samples_leaf=3,max_features='auto', bootstrap=True, n_jobs=-1)
nskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=1)
scores = cross_val_score(rfc, normalized_INPUT, Y, scoring='f1_macro', verbose=4,cv=nskf, n_jobs=-1)
scores
# report performance
print('f1_macro: %.3f (%.3f)' % (mean(scores), std(scores)))
print('\nMaximum f1 score: ', max(scores))
print('\nMinimum f1 score: ', min(scores))


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done  48 out of  50 | elapsed:    4.4s remaining:    0.1s


f1_macro: 0.805 (0.074)

Maximum f1 score:  0.9410919540229885

Minimum f1 score:  0.6000000000000001


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    4.7s finished


In [102]:
#classification report 
originalclass = []
predictedclass = []
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

X = normalized_INPUT
y = Y

def classification_report_with_f1_score(y_train, y_test):
    originalclass.extend(y_train)
    predictedclass.extend(y_test)
    return f1_score(y_train, y_test,average='macro') # return accuracy score

classif_report = cross_val_score(model2, normalized_INPUT,Y, cv=nskf, scoring=make_scorer(classification_report_with_f1_score))

# Average values in classification report for all folds in a K-fold Cross-validation  
print(classification_report(originalclass, predictedclass))

              precision    recall  f1-score   support

           0       0.92      0.93      0.92      1490
           1       0.79      0.78      0.79       530

    accuracy                           0.89      2020
   macro avg       0.86      0.85      0.85      2020
weighted avg       0.89      0.89      0.89      2020



In [99]:
model2.fit(normalized_INPUT,Y)
model2.predict(normalized_INPUT)

array([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0,
       1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1], dtype=int64)

In [100]:
model2.score(normalized_INPUT, Y)

0.9653465346534653